# Get Vasari letters from Fondazione Memofonte (https://www.memofonte.it/ricerche/giorgio-vasari/) with Beautiful Soup

#### based on Jeri Wieringa, "Intro to Beautiful Soup," The Programming Historian 1 (2012), https://doi.org/10.46430/phen0008.

In [1]:
import urllib.request, urllib.error, urllib.parse
import csv
from bs4 import BeautifulSoup
import html2text
import os
from datetime import datetime
import numpy as np
import pandas as pd

In [2]:
# function to check if 'nan'

def isnan(x):
    return x != x
    

In [3]:
data_path = 'data/results/letters/'

## Iterate over all URLs and save as .html

#### make list of URLs

In [2]:
# letter-id ranging from 1 to 1151

url_list = []
url_base = "https://www.memofonte.it/home/ricerca/singolo_17.php?id="
for i in range (1151):
    url_list.append(url_base+ str(i+1))
    
len(url_list)

1151

### save all as .html

In [40]:
count = 0
for i in url_list:
    count+=1
    response = urllib.request.urlopen(i)
    webContent = response.read()
    #filename = 'data/results/letters/html/letter-'+str(i)'.html'
    f = open('data/results/letters/html/letter-'+str(count)+'.html', 'wb')
    f.write(webContent)
    f.close()
    

## Open all .html, make Beautiful Soups, extract information, save all letters in a single CSV

In [73]:
list_of_htmls = os.listdir('data/results/letters/html')
len(list_of_htmls)
#list_of_htmls

1151

In [108]:
# obo.py
# get rid of tags without Beautiful Soup
# because there is a space " " missing after some <br/> tags

def stripTags(pageContent):
    pageContents = str(pageContent)

    inside = 0
    text = ''
    storage = " "

    for char in pageContents:
        if (char == '<' and storage == " "):
            inside = 1
        elif (char == '<' and storage != " "):
            inside = 1
            text += " "     
        elif (inside == 1 and char == '>'):
            inside = 0
            #text += " "
        elif inside == 1:
            continue
        else:
            text += char
        storage = char

    return text

In [127]:
# test code and output for single file before iterating over 1151

csv_row = list()

soup = BeautifulSoup(open('data/results/letters/html/'+str(list_of_htmls[0])), "lxml")

p_array = soup.find_all('p')

csv_row.append(int(p_array[0].text[(len('Numero d\'ordine:')+1):]))
csv_row.append(((p_array[1].text[(len('Data:')+1):])))
csv_row.append(p_array[2].text[(len('Intestazione:')+1):])
csv_row.append(p_array[3].text[(len('Segnatura:')+1):])

#csv_row.append(p_array[4].text[(len('Fonte:')+1):]) # the implemented .text function is not fitting the text
# extra lines for text at p_array[4]
x = (p_array[4])
formatted_text = stripTags(str(x)[12:]).replace("\xa0", '')

csv_row.append(formatted_text)
csv_row.append(p_array[5].text[(len('Bibliografia:')+1):])

# preprocess the text because \n is replaced with nothing "" - test solution: replace with space " "
csv_row

[1051,
 '26 12  1572',
 'VINCENZO BORGHINI IN FIRENZE A GIORGIO VASARI IN ROMA',
 'ASA, AV, 14 (XLVIII), cc. 146-148.',
 'Molto Magnifico messer Giorgio. Ho ricevuto la vostra con la distesa delle istorie della sala, le quali considererò, e perché mi ha dato noia un po’ di scesa et non ne son libero, indugierò a questa altra settimana a darne più piena risposta. Messer Vincenzo non credo che pensi più alla messa, ma quando si potessi confermare nel privileggio del legittimare et altri indulti che non toccono il concilio, credo sarebbe contento, che qua non lo ammettono, e però forse bisognerebbe questa confermazione; pure ci sarà tempo. De la cosa mia come mi dite ne starò a buona speranza; e se voi vedete che la camini bene, il meglio sarebbe stringnere e darli fine, che cosa fatta importa assai e si schifa molte cose, che col tempo possono occorrere, che si dice che l’indugio pigla vizio; ma se ci fusse qualche difficultà, allora è bene non strignere, che il far male ha da essere l’u

In [34]:
# inspect data
#soup.prettify

In [70]:
# test how to convert the date

datetime_object = datetime.strptime((p_array[1].text[(len('Data:')+1):]), "%d %m %Y")
datetime_object

datetime.datetime(1572, 12, 26, 0, 0)

### Make one CSV File for all letters

In [129]:
# make CSV File
# changed filename after first run from letters.csv to letters_2.csv

csv_file = csv.writer(open('data/results/letters/letters_2.csv', "w"))

# Write column headers as the first line
csv_file.writerow(["Numero d'ordine", "Data", "Intestazione", "Segnatura", "Fonte", "Bibliografia"]) 

64

### iterate over all .html files, save as csv

In [130]:
# open all letter.html, make soup, extract information, collect it in a list, append list as a row to csv_file

for i in list_of_htmls:
    csv_row = list() # make new row
    soup = BeautifulSoup(open('data/results/letters/html/'+i), "lxml") # make soup
    p_array = soup.find_all('p') # take the information one by one 
    csv_row.append(int(p_array[0].text[(len('Numero d\'ordine:')+1):])) # append each to the list
    csv_row.append(p_array[1].text[(len('Data:')+1):])
    csv_row.append(p_array[2].text[(len('Intestazione:')+1):])
    csv_row.append(p_array[3].text[(len('Segnatura:')+1):])
    #csv_row.append(p_array[4].text[(len('Fonte:')+1):])
    x = (p_array[4])
    formatted_text = stripTags(str(x)[12:]) # remove html tags without beautiful soup
    csv_row.append(formatted_text.replace("\xa0", ''))
    
    csv_row.append(p_array[5].text[(len('Bibliografia:')+1):])
    csv_file.writerow(csv_row)


## open CSV-file as dataframe

In [131]:
filename = 'data/results/letters/letters_2.csv'
csv_file = csv.reader(open(filename, "r"))
dataframe = pd.read_csv(filename)
#dataframe

dataframe["Fonte"][1146]

'Reverendissimo monsignore patron mio, io mando alla Signoria Vostra Reverendissima questa inclusa ch’è della Comunità d’Arezzo; la Signoria Vostra sia contenta, spedita ch’ella sarà, di rimandamela in mano, perché importa alla mia Comunità e ve ne arò obligo. Io vi ho scritto e Sua Eccellenzia doverrà deliberare quanto gli piacerà delle cose che bosogniano per qua per servizio suo. Ebbi un’ultima sua col sugel grosso, farò quanto m’è comesso e tuttavia si atende; così potessi voi con tutta la corte goder quel ch’è fatto senza avere a star ne’ luoghi che son disfatti, pure il faro che vi conduce lo debbe saper lui; andrò pregando Dio che vi riconduca a noi e che vi dia pazienzia che questo santo ch’è oggi nel giorno, che io scrivo, i diavoli gniene scossono coi bastoni; dove il nostro Bernia dice in questi due versi: a giorni diciassette l’Alcionio fu bastonato come santo Antonio. E con questo fo fine, restando al suo comando. Di vostra Signoria reverendissimo servitore Giorgio Vasari 

## get rid of nan-values

'nan' count in total: 11 rows with nan values 
--> replace with '0' and collect index for documentation (row-indices found: 17, 101, 268, 368, 491, 569, 617, 703, 740, 928, 985)

In [132]:
count = 0
for i in range(len(dataframe)): # check column by column if isnan and replace with '0'
    
    if isnan(dataframe['Intestazione'][i]): 
        print('Intestazione, ', count)
        (dataframe['Intestazione'][i]) = '0'
        
    if isnan(dataframe['Segnatura'][i]):
        #print('Segnatura', count)
        (dataframe['Segnatura'][i]) = '0'
        
    if isnan(dataframe['Fonte'][i]):
        #print('Fonte', count)
        (dataframe['Fonte'][i]) = '0'
        
    if isnan(dataframe['Bibliografia'][i]):
        #print('Bibliografia', count)
        (dataframe['Bibliografia'][i]) = '0'
    
    
    count+=1
            

Intestazione,  17
Intestazione,  101
Intestazione,  268
Intestazione,  368
Intestazione,  491
Intestazione,  569
Intestazione,  617
Intestazione,  703
Intestazione,  740
Intestazione,  928
Intestazione,  985


<ipython-input-132-f6e2de138291>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  (dataframe['Intestazione'][i]) = '0'
<ipython-input-132-f6e2de138291>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  (dataframe['Segnatura'][i]) = '0'
<ipython-input-132-f6e2de138291>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  (dataframe['Bibliografia'][i]) = '0'


# Distinguish letters written by Vasari himself from copies of his nefew

example for signature of letter copied by nefew (called Giorgio Vasari the younger) 
BRF, 2354, cc. 12v-14, copia di Giorgio Vasari il Giovane.

number of letters filtered at http://vasariscrittore.memofonte.it/archivio
390

In [152]:
Vasari = list() # list for letters sent by Vasari
Vasari_no_copies = list() # filter out the copies of his nefew
copies_only = list()
#Va_indices = list()



#    if dataframe['Intestazione'][i].startswith('GIORGIO VASARI') or dataframe['Intestazione'][i].startswith(' GIORGIO VASARI'): # VGiorgio VASARI
count = 0
for i in range(len(dataframe)):
    count +=1 
    # all letters Vasari sent
    if dataframe['Intestazione'][i].startswith('GIORGIO VASARI') or dataframe['Intestazione'][i].startswith(' GIORGIO VASARI'): # GIORGIO VASARI
        #print(dataframe['Intestazione'][i])
        #print(dataframe['Segnatura'][i], count)
        Vasari.append(list(dataframe.iloc[i]))
    # copies filtered out
    if (dataframe['Intestazione'][i].startswith('GIORGIO VASARI') or dataframe['Intestazione'][i].startswith(' GIORGIO VASARI')) and 'copia' not in dataframe['Segnatura'][i]: # filter out copies
        #print(dataframe['Segnatura'][i], count)
        Vasari_no_copies.append(list(dataframe.iloc[i]))
    # copies filtered in
    if (dataframe['Intestazione'][i].startswith('GIORGIO VASARI') or dataframe['Intestazione'][i].startswith(' GIORGIO VASARI')) and 'copia' in dataframe['Segnatura'][i]: # filter in copies
        #print(dataframe['Segnatura'][i], count)
        copies_only.append(list(dataframe.iloc[i]))

        
len(Vasari), len(Vasari_no_copies), len(copies_only)

(404, 363, 41)

In [153]:
# check
'copia' in dataframe['Segnatura'][29]

True

Make new dataframes and save as csv files

In [154]:
columns = ["Numero d'ordine", "Data", "Intestazione", "Segnatura", "Fonte", "Bibliografia"]
data_path = 'data/results/letters/'

In [155]:
df_Vasari = pd.DataFrame(Vasari, columns = columns)
df_Vasari_no_copies = pd.DataFrame(Vasari_no_copies, columns = columns)
df_copies_only = pd.DataFrame(copies_only, columns = columns)

In [156]:
df_Vasari.to_csv(data_path + 'Vasari-all.csv', index = False, header=True)
df_Vasari_no_copies.to_csv(data_path + 'Vasari-no_copies.csv', index = False, header=True)
df_copies_only.to_csv(data_path + 'Vcopies.csv', index = False, header=True)


# open Vasari csv-letter-files as dataframes

In [157]:
data_path = 'data/results/letters/'
Vasari_all = 'Vasari-all.csv'
Vasari_only = 'Vasari-no_copies.csv'
copies_only = 'Vcopies.csv'

df_Vasari_all = pd.read_csv(data_path+Vasari_all)
df_Vasari_only = pd.read_csv(data_path+Vasari_only)
df_copies_only = pd.read_csv(data_path+copies_only)

## split dataframe of letters for different involved authorship candidates

Cosimo Bartoli and Pierfrancesco Giambullari involved as Co-Authors after Gerd Blum (https://archiv.ub.uni-heidelberg.de/artdok/1948/1/Blum_Provvidenza_e_progresso_2010.pdf) 

Pierfrancesco Giambullari, Cosimo Bartoli, Giovan Battista Gelli und Carlo Lenzoni involved in the preparation of the edition of 1550; massive contribution of Vincenzio Borghinis concerning the indices and growing involvement over the following decennies (Alessandro Nova: Vasari versus Vasari, S.151)

In [139]:
# get Pierfrancesco Giambullari, Cosimo Bartoli, Vincenzio Borghini
# examples of the signatures copied from http://vasariscrittore.memofonte.it/archivio# 
# COSIMO BARTOLI IN FIRENZE A GIORGIO VASARI IN ROMA (73 letters)
# VINCENZO BORGHINI IN FIRENZE A GIORGIO VASARI IN ROMA (131 letters)
# PIERFRANCESCO GIAMBULLARI IN FIRENZE A GIORGIO VASARI IN AREZZO (6 letters)

Borghini = list() # collect whole entries written by Vincenzo Borghini
Bo_indices = list() # collect indices of Borghini entries

Giambullari = list() # collect Pierfrancesco Giambullari
G_indices = list()

Bartoli = list() # collect Pierfrancesco Giambullari
Ba_indices = list()

for i in range(len(dataframe)):

    
    if dataframe['Intestazione'][i].startswith('VINCENZO BORGHINI') or dataframe['Intestazione'][i].startswith(' VINCENZO BORGHINI'): # VINCENZO BORGHINI
        Borghini.append(list(dataframe.iloc[i]))
    
    if 'COSIMO BARTOLI' in dataframe['Intestazione'][i]: # COSIMO BARTOLI
        Bartoli.append(list(dataframe.iloc[i]))
    
    if 'PIERFRANCESCO GIAMBULLARI' in dataframe['Intestazione'][i]:
        Giambullari.append(list(dataframe.iloc[i]))
        
    count += 1
    
    
print('Giambullari: ', len(Giambullari), 'Borghini: ', len(Borghini), 'Bartoli: ', len(Bartoli))

Giambullari:  6 Borghini:  136 Bartoli:  73


## make single dataframes for Borghini, Bartoli and Giambullari

In [140]:
columns = ["Numero d'ordine", "Data", "Intestazione", "Segnatura", "Fonte", "Bibliografia"]

In [141]:
df_Giambullari = pd.DataFrame(Giambullari, columns = columns)
df_Borghini = pd.DataFrame(Borghini, columns = columns)
df_Bartoli = pd.DataFrame(Bartoli, columns = columns)

In [142]:
df_Borghini

,Numero d'ordine,Data,Intestazione,Segnatura,Fonte,Bibliografia
0,1051,26 12 1572,VINCENZO BORGHINI IN FIRENZE A GIORGIO VASARI ...,"ASA, AV, 14 (XLVIII), cc. 146-148.",Molto Magnifico messer Giorgio. Ho ricevuto la...,"Frey 1930, pp. 736-737."
1,512,8 06 1564,VINCENZO BORGHINI IN GLI ALBERI A GIORGIO VAS...,"ASA, AV, 14 (XLVIII), c. 41.",Magnifico Messer Giorgio. Io mi muoio di sonno...,"Frey 1930, p. 84."
2,912,22 09 1571,VINCENZO BORGHINI IN FIRENZE A GIORGIO VASARI ...,"ASA, AV, 14 (XLVIII), c. 101.",Molto Magnifico Signor Cavalier. Il Gabburra a...,"Frey 1930, pp. 607-608."
3,302,3 03 1560,VINCENZO BORGHINI IN POPPIANO A GIORGIO VASARI...,"ASA, AV, 14 (XLVIII), cc. 195-198.",Magnifico messer Giorgio. Come io son giunto i...,"Frey 1923, pp. 534-535."
4,617,6 05 1566,VINCENZO BORGHINI IN POPPIANO A GIORGIO VASARI...,"ASA, AV, 14 (XLVIII), cc. 219-220.",Molto Magnifico messer Giorgio. Trovandomi a P...,"Frey 1930, pp. 236-238."
...,...,...,...,...,...,...
131,602,1 01 1565,VINCENZO BORGHINI IN FIRENZE A GIOVANNI CACCIN...,"ASA, AV, 14 (XLVIII), c. 181.",Messer Giovanni. Se e’ non si pensa al canto a...,"Frey 1930, p. 217 (fine novembre-inizio dicemb..."
132,897,1 01 1571,VINCENZO BORGHINI IN FIRENZE A GIORGIO VASARI ...,"ASA, AV, 14 (XLVIII), c. 176.",Magnifico Signor mio. Non occorre dir cosa alc...,"Frey 1930, pp. 594-595 (fine luglio)."
133,751,15 05 1569,VINCENZO BORGHINI IN FIRENZE A GIORGIO VASARI ...,"ASPI, AOST, Suppliche ed Informazioni, Filza I...",Messer Giorgio onorando. Ho considerata questa...,"Frey 1930, pp. 423-424."
134,1091,8 05 1573,VINCENZO BORGHINI IN FIRENZE A GIORGIO VASARI ...,"ASA, AV, 14 (XLVIII), cc. 161-182.",Molto Magnifico messer Giorgio mio. Ho la vost...,"Frey 1930, pp. 783-784."


## save dataframes as csv files

In [143]:
data_path = 'data/results/letters/'

In [144]:
df_Giambullari.to_csv(data_path + 'Giambullari.csv', index = False, header=True)
df_Borghini.to_csv(data_path + 'Borghini.csv', index = False, header=True)
df_Bartoli.to_csv(data_path + 'Bartoli.csv', index = False, header=True)

### open Borghini, Giambullari and Bartoli letter-csv-files as dataframes

In [145]:
data_path = 'data/results/letters/'
Gia = 'Giambullari.csv'
Bor = 'Borghini.csv'
Bar = 'Bartoli.csv'

df_Giambullari = pd.read_csv(data_path+Gia)
df_Borghini = pd.read_csv(data_path+Bor)
df_Bartoli = pd.read_csv(data_path+Bar)

# make files for DANTE code

for this GitHub Repo (https://github.com/AlexMoreo/dante-verification) 
the documents must be named "author_texname.txt"

In [146]:
#df_Giambullari = pd.read_csv(data_path+Gia)
#df_Borghini = pd.read_csv(data_path+Bor)
#df_Bartoli = pd.read_csv(data_path+Bar)
df_Vasari_all = pd.read_csv(data_path+Vasari_all)
#df_Vasari_only = pd.read_csv(data_path+Vasari_only)

In [147]:
df_Vasari_all["Fonte"][401]

'Reverendissimo monsignore patron mio, io mando alla Signoria Vostra Reverendissima questa inclusa ch’è della Comunità d’Arezzo; la Signoria Vostra sia contenta, spedita ch’ella sarà, di rimandamela in mano, perché importa alla mia Comunità e ve ne arò obligo. Io vi ho scritto e Sua Eccellenzia doverrà deliberare quanto gli piacerà delle cose che bosogniano per qua per servizio suo. Ebbi un’ultima sua col sugel grosso, farò quanto m’è comesso e tuttavia si atende; così potessi voi con tutta la corte goder quel ch’è fatto senza avere a star ne’ luoghi che son disfatti, pure il faro che vi conduce lo debbe saper lui; andrò pregando Dio che vi riconduca a noi e che vi dia pazienzia che questo santo ch’è oggi nel giorno, che io scrivo, i diavoli gniene scossono coi bastoni; dove il nostro Bernia dice in questi due versi: a giorni diciassette l’Alcionio fu bastonato come santo Antonio. E con questo fo fine, restando al suo comando. Di vostra Signoria reverendissimo servitore Giorgio Vasari 

In [11]:
#df_Giambullari

In [148]:
Dante_path = 'data/results/Dante/'
author = 'Giambullari'                      # Giambullari .txt files

for i in range(len(df_Giambullari)):
    
     # get text, letter-number and create filename
    text = df_Giambullari["Fonte"][i] 
    number = str(df_Giambullari["Numero d'ordine"][i]) 
    filename = Dante_path + author + '_' + number + '.txt'
    
     # write .txt file
    output = open(filename, "wt")
    output.write(text)
    output.close()


In [149]:
Dante_path = 'data/results/Dante/'
author = 'Borghini'                      # Borghini .txt files

for i in range(len(df_Borghini)):
    
     # get text, letter-number and create filename
    text = df_Borghini["Fonte"][i] 
    number = str(df_Borghini["Numero d'ordine"][i]) 
    filename = Dante_path + author + '_' + number + '.txt'
    
     # write .txt file
    output = open(filename, "wt")
    output.write(text)
    output.close()


In [150]:
Dante_path = 'data/results/Dante/'
author = 'Bartoli'                      # Bartoli .txt files

for i in range(len(df_Bartoli)):
    
     # get text, letter-number and create filename
    text = df_Bartoli["Fonte"][i] 
    number = str(df_Bartoli["Numero d'ordine"][i]) 
    filename = Dante_path + author + '_' + number + '.txt'
    
     # write .txt file
    output = open(filename, "wt")
    output.write(text)
    output.close()


In [151]:
Dante_path = 'data/results/Dante/'
author = 'Vasari'                      # Vasari.txt files  ! only Vasari_only letters!

for i in range(len(df_Vasari_only)):
    
     # get text, letter-number and create filename
    text = df_Vasari_only["Fonte"][i] 
    number = str(df_Vasari_only["Numero d'ordine"][i]) 
    filename = Dante_path + author + '_' + number + '.txt'
    
     # write .txt file
    output = open(filename, "wt")
    output.write(text)
    output.close()


In [158]:
Dante_path = 'data/results/Dante_copies/'
author = 'Vcopy'                      # Vcopy.txt files  ! only copies from his nephew

for i in range(len(df_copies_only)):
    
     # get text, letter-number and create filename
    text = df_Vasari_only["Fonte"][i] 
    number = str(df_Vasari_only["Numero d'ordine"][i]) 
    filename = Dante_path + author + '_' + number + '.txt'
    
     # write .txt file
    output = open(filename, "wt")
    output.write(text)
    output.close()


### make unkown_files

In [50]:
source_path = 'data/Italian-version/file-per-artist-italian/'
unknown_path = 'data/results/Dante_unkown/'


#### make new filenames

In [63]:
list_of_files = os.listdir('data/Italian-version/file-per-artist-italian')
renamed_list = list()

for i in list_of_files:
    renamed_list.append(unknown_path + 'unkown_' + (i.replace(' ', '-').replace("'", "").replace(",","-").strip('_').split('_'))[1] + '.txt')

    

#len(renamed_list), len(list_of_files)
#list_of_files

len(renamed_list), len(list_of_files), renamed_list[7]

(168, 168, 'data/results/Dante_unkown/unkown_Francesco-Francia.txt')

In [56]:
# load file
as_input = open((source_path + list_of_files[0]), 'r')
# create filename
filename = renamed_list[0] 
# take text
#text = as_input.read()
 # write .txt file
#output = open(filename, "wt")
#output.write(text)
#+output.close()
filename
#as_input

'data/results/Dante_unkown/unkown_Lorenzo Costa.txt'

In [64]:
# build iterator for all biographies
index = 0
for i in list_of_files:
    # open file
    as_input = open((source_path + i), 'r')
    # take text
    text = as_input.read()
    as_input.close
    
    output = open(renamed_list[index], "wt")
    output.write(text)
    output.close()
    print(renamed_list[index])
    index += 1
    
    # write .txt file

data/results/Dante_unkown/unkown_Lorenzo-Costa.txt
data/results/Dante_unkown/unkown_Taddeo-Gaddi.txt
data/results/Dante_unkown/unkown_Mariotto-Albertinelli.txt
data/results/Dante_unkown/unkown_Fra-GiovannAgnolo-Montorsoli.txt
data/results/Dante_unkown/unkown_Giovanfrancesco-Rustichi.txt
data/results/Dante_unkown/unkown_Ridolfo--Davit-e-Benedetto-Grillandai.txt
data/results/Dante_unkown/unkown_GiovannAntonio-Lappoli.txt
data/results/Dante_unkown/unkown_Francesco-Francia.txt
data/results/Dante_unkown/unkown_Auteur.txt
data/results/Dante_unkown/unkown_Agostino-et-Agnolo.txt
data/results/Dante_unkown/unkown_Niccolò-detto-il-Tribolo.txt
data/results/Dante_unkown/unkown_Marco-Calavrese.txt
data/results/Dante_unkown/unkown_Francesco-Mazzuoli.txt
data/results/Dante_unkown/unkown_Giovanni-Antonio-Sogliani.txt
data/results/Dante_unkown/unkown_Masolino.txt
data/results/Dante_unkown/unkown_Duccio.txt
data/results/Dante_unkown/unkown_Giovan-Francesco-detto-il-Fattore-e-Pellegrino-da-Modana.txt
data

In [58]:
os.listdir('data/results/Dante/')

['Vasari_509.txt',
 'Vasari_535.txt',
 'Vasari_253.txt',
 'Vasari_247.txt',
 'Borghini_302.txt',
 'Borghini_666.txt',
 'Bartoli_948.txt',
 'Borghini_909.txt',
 'Vasari_864.txt',
 'Vasari_680.txt',
 'Vasari_858.txt',
 'Vasari_469.txt',
 'Vasari_441.txt',
 'Vasari_333.txt',
 'Vasari_455.txt',
 'Bartoli_237.txt',
 'Vasari_1034.txt',
 'Vasari_482.txt',
 'Vasari_496.txt',
 'Vasari_497.txt',
 'Vasari_1021.txt',
 'Vasari_1035.txt',
 'Vasari_52.txt',
 'Vasari_332.txt',
 'Vasari_440.txt',
 'Vasari_326.txt',
 'Vasari_681.txt',
 'Vasari_865.txt',
 'Vasari_656.txt',
 'Vasari_905.txt',
 'Borghini_897.txt',
 'Borghini_868.txt',
 'Borghini_840.txt',
 'Vasari_291.txt',
 'Vasari_246.txt',
 'Vasari_252.txt',
 'Borghini_498.txt',
 'Vasari_244.txt',
 'Vasari_250.txt',
 'Borghini_842.txt',
 'Borghini_856.txt',
 'Vasari_132.txt',
 'Vasari_126.txt',
 'Bartoli_744.txt',
 'Bartoli_750.txt',
 'Vasari_456.txt',
 'Vasari_442.txt',
 'Vasari_50.txt',
 'Vasari_1023.txt',
 'Bartoli_220.txt',
 'Vasari_1037.txt',
 'Vas

### Helper Funcitons

In [17]:
# get rid of strange line break "\xa0"
# open a file and overwrite it

def fix(name):
    """ Fix a single .txt file
    """
    input = open(name, "rt")
    text = input.read()
    input.close()
    text = text.decode("utf-8")

    # Show if we get bad hits
    for c in text:
        if c == u"\xa0":
            print("Ufff")

    text = text.replace(u"\xa0", u" ")
    text = text.encode("utf-8")

    output = open(name, "wt")
    output.write(text)
    output.close()


# Process all .txt files in the
# current folder
for f in os.listdir(os.getcwd()):
    if f.endswith(".txt"):
        fix(f)

In [104]:
# obo.py
# get rid of tags without Beautiful Soup
# because there is a space " " missing after some <br/> tags

def stripTags(pageContent):
    pageContents = str(pageContent)

    inside = 0
    text = ''

    for char in pageContents:
        if char == '<':
            inside = 1
        elif (inside == 1 and char == '>'):
            inside = 0
            #after = 1
            #text += " "
        elif inside == 1:
            continue
        else:
            text += char

    return text